# Generate Tests

This notebook imports a compiled ThermoCodegen object and uses the `Tester` class to generate pandas dataframes
and pytest files for testing all of the interfaces to the code generated objects


This notebook will generate tests for both the pure Mg endmember system and the solid-solution MgFe system

To get pure phases, just set the variable `MODEL='pure'` else `MODEL='solution'`

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from glob import glob
import pytest

## Import the Tester Class from thermocodegen

In [ ]:
from thermocodegen.testing import Tester

Set a time-stamp for the tests


In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

#### Decide on pure or solution phase model

In [ ]:
MODEL = 'pure'
#MODEL = 'solution'
import thermocodegen as tcg
version = tcg.__version__
print("using {} phases model for thermocodegen v{}".format(MODEL,version))

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath('../')
TEST_DIR = HOME_DIR+'/tests/test-v{}_{}_rxn_{}'.format(version,MODEL,timestamp)
try:
    os.makedirs(TEST_DIR)
except:
    pass
print(TEST_DIR)

### Import the thermocodegen reaction module 
make sure that the module is available in your python path

In [ ]:
# load reaction object
if MODEL == 'pure':
    import py_Mg2SiO4_stixrude as pms
    rxn = pms.Mg2SiO4_stixrude()
elif MODEL == 'solution':
    import py_MgFeSiO4_stixrude as pms
    rxn = pms.MgFeSiO4_stixrude()


In [ ]:
rxn.report()

In [ ]:
pms.phase_info()

### Create a tester object
and report the phase dictionary

In [ ]:
tester = Tester.factory(pms)
tester.phase_dict

## Test endmembers

### set input dataframe for endmembers

In [ ]:
em_df_in = tester.get_endmember_df_in()
em_df_in

### just set P,T fields for testing

In [ ]:
# Set test variables
T=1700.
P=100000.
em_df_in['P']= P*np.ones(len(em_df_in))
em_df_in['T']= T*np.ones(len(em_df_in))
em_df_in.dropna(axis=1,inplace=True)
em_df_in

### Generate output testing dataframe

In [ ]:
em_df_out = tester.set_df_out(em_df_in)
em_df_out

### Output dataframe as csv file and pytest files to TEST_DIR

In [ ]:
em_df_out.to_csv('{}/Endmembers.csv'.format(TEST_DIR))

In [ ]:
fileroot='{}/test_endmembers'.format(TEST_DIR) 
tester.write_tests(em_df_out,fileroot=fileroot)

### run pytest on the test files

In [ ]:
file = glob(fileroot+'*')[-1]
pytest.main(['--disable-warnings',file])

## Test Phases
### set input dataframe for phases

In [ ]:
ph_df_in = tester.get_phase_df_in()
ph_df_in

### just set P,T,c,x,n,e fields for testing

In [ ]:
rxn.report()

In [ ]:
# Set test variables
T=1700.
P=100000.
i_c = 0
pure_phases = ['Forsterite', 'MgWadsleyite', 'MgRingwoodite', 'Fayalite', 'FeWadsleyite', 'FeRingwoodite', 'MgPerovskite', 'Periclase', 'FePerovskite', 'Wuestite']
soln_phases = [ 'Olivine', 'Wadsleyite', 'Ringwoodite']

### Set concentrations for pure and solution phase

In [ ]:
x_pure = [1.]
x_soln = [0.9, 0.1]
c_pure = pms.Forsterite().x_to_c(x_pure)
c_soln = pms.Olivine().x_to_c(x_soln)

In [ ]:
N = len(ph_df_in)
ph_df_in['P']= P*np.ones(N)
ph_df_in['T']= T*np.ones(N)
for index,row in ph_df_in.iterrows():
    name = row['name']
    if name in pure_phases:
        print('Pure phase:', index,name)
        ph_df_in['c'][index] = np.array(c_pure)
        ph_df_in['x'][index] = np.array(x_pure)
        ph_df_in['n'][index] = np.array(x_pure)
        ph_df_in['e'][index] = getattr(pms,name)().conv_moles_to_elm(x_pure)
    elif name in soln_phases:
        print('Soln phase:', index,name)
        ph_df_in['c'][index] = np.array(c_soln)
        ph_df_in['x'][index] = np.array(x_soln)
        ph_df_in['n'][index] = np.array(x_soln)
        ph_df_in['e'][index] = getattr(pms,name)().conv_moles_to_elm(x_soln)
ph_df_in['i']=i_c
ph_df_in.dropna(axis=1,inplace=True)
ph_df_in

### Generate output testing dataframe and pytest files

In [ ]:
ph_df_out = tester.set_df_out(ph_df_in)
ph_df_out

In [ ]:
ph_df_out.to_csv('{}/Phases.csv'.format(TEST_DIR))

In [ ]:
fileroot='{}/test_phases'.format(TEST_DIR) 
tester.write_tests(ph_df_out,fileroot=fileroot)

### run pytest on the test files

In [ ]:
file=glob(fileroot+'*')[-1]
pytest.main(['--disable-warnings',file])

## Test Reactions 
### set input dataframe for reactions

In [ ]:
rx_df_in = tester.get_reaction_df_in()
rx_df_in

### just set P,T,c,x,n fields for testing

In [ ]:
rxn.report()

In [ ]:
# Set test variables
C = rxn.zero_C()
if MODEL == 'pure':
    for i in range(len(C)):
        C[i] = c_pure
else:
    for i in range(len(C)):
        C[i] = c_soln
X = rxn.C_to_X(C)
Phi = [0.33, 0.33, 0.33]

# Variables for parameter set and get functions
p, val = "R", 8.31442

#jacobian terms
j = 0
i = 1
l = 0
k = 0
m = 1

In [ ]:
rx_df_in['P']= P
rx_df_in['T']= T
rx_df_in['C']= [C]
rx_df_in['X']= [X]
rx_df_in['Phi'] = [Phi]
rx_df_in['p'] = p
rx_df_in['val'] = val
rx_df_in[['i','j','k','l','m']]=pd.DataFrame.from_dict(dict(i=[i],j=[j],k=[k],l=[l],m=[m]))
rx_df_in


### Generate output testing dataframe and pytest files

In [ ]:
rx_df_out = tester.set_df_out(rx_df_in)
rx_df_out

In [ ]:
rx_df_out.to_csv('{}/Reactions.csv'.format(TEST_DIR))

In [ ]:
fileroot='{}/test_rxns'.format(TEST_DIR)
tester.write_tests(rx_df_out,fileroot=fileroot)

### run pytest on the test files

In [ ]:
file = glob(fileroot+'*')[-1]
pytest.main(['--disable-warnings',file])